# First Steps Connecting to the Wereldculturen MS SQL Server

Using pyodbc to connect to the server and submit queries to the DB


Can also use Pandas to directly parse query result into DataFrame (via `pandas.read_sql`)

In [19]:
import pandas as pd
import numpy as np

In [2]:
import pyodbc 
server = 'tcp:azuredfserv.database.windows.net' 
database = 'Azuredf' 
username = 'Demouser' 
password = 'Knxdde#77' 
driver='{ODBC Driver 17 for SQL Server}'

In [3]:
def table_to_DataFrame(connection, table_name, until=None):
    if not until:
        until = ""
    query = f"SELECT TOP {until} * FROM {table_name};"  # = f"SELECT * FROM {table_name} TABLESAMPLE ({until} ROWS);"
    df = pd.read_sql(query, connection)
    return df


def connect_to_DB():
    return pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

In [ ]:
with connect_to_DB() as conn:
    objects = table_to_DataFrame(conn, "Objects", until=5000)

In [ ]:
for col in objects.columns:
    vals = objects[col]
    
    print(col)
    print(len(vals.unique()))
#     print(vals.unique())

print()

In [ ]:
objects.apply(lambda col: len(col.unique()), axis=0)

In [ ]:
objects.TextSearchID.unique()

In [ ]:
with connect_to_DB() as conn:
    query = f"SELECT [ObjectID] FROM Objects;"
    df = pd.read_sql(query, conn)


# 0. Make Connection and List Databases on the Server

In [ ]:
with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT name, collation_name FROM sys.databases")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()
            

# 1. Get List of Tables in the Database

In [ ]:
with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("""select schema_name(t.schema_id) as schema_name,
                               t.name as table_name,
                               t.create_date,
                               t.modify_date
                               from sys.tables t""")
        
        
        for row in cursor.fetchall():
            print(row)
            print()

        
        cursor.execute("""select t.name
                               from sys.tables t""")

        
        table_names = cursor.fetchall()
        print(table_names)

In [ ]:
cursor.execute("""select * from sys.tables t""")

cursor.fetchall()

In [ ]:
for tbl in table_names:
    n = tbl[0]
    cursor.execute(f"""SELECT COLUMN_NAME 
                        FROM INFORMATION_SCHEMA.COLUMNS
                        WHERE TABLE_NAME = '{n}' AND TABLE_SCHEMA='dbo'""")
    columns = cursor.fetchall()
            
            
    cursor.execute(f"select TOP 10 * from {n} ;")
    table = cursor.fetchall()
#            print(n, "has entries : ", len(cursor.fetchall()))
#     print(n)
#     print()
#     print(columns)
#     print()
#     print(table)
    
    
    df = pd.DataFrame.from_records(table, columns=[tup[0] for tup in columns])
   
    try:
        df["GSRowVersion"] = df["GSRowVersion"].str.decode("iso-8859-1")  # df.apply(lambda row: str.encode(row["GSRowVersion"]))
    except KeyError:
        pass
    
    print(n)
    print(df.head())
            
    print("\n", "-"*30, "\n\n")

## Test Random SELECT

In [39]:
# ObjectID

rands = np.random.randint(0, 1000, size=10)
rands = ",".join(map(str, np.sort(rands)))
print(rands)

# q = "SELECT MAX(DepartmentID) AS maxid FROM Departments"

q = f"SELECT ObjectID FROM Objects WHERE ObjectID IN ({rands});"

m = 100
q = f"SELECT ObjectID FROM Objects TABLESAMPLE ({10000} ROWS);"

122,258,351,360,466,693,781,856,915,927


In [40]:
with connect_to_DB() as conn:
    with conn.cursor() as cursor:
        cursor.execute(q)
        for row in cursor.fetchall():
            print(row)
            print()

(248, )

(249, )

(250, )

(251, )

(810, )

(811, )

(812, )

(813, )

(814, )

(815, )

(816, )

(817, )

(818, )

(2284, )

(2285, )

(2286, )

(2287, )

(4436, )

(4438, )

(4440, )

(4442, )

(4444, )

(4446, )

(4448, )

(585787, )

(585788, )

(585789, )

(585790, )

(585826, )

(590726, )

(607064, )

(608866, )

(608867, )

(662532, )

(662533, )

(662534, )

(662535, )

(662536, )

(662537, )

(662538, )

(665352, )

(665353, )

(665354, )

(665355, )

(665356, )

(665357, )

(665358, )

(665359, )

(665360, )

(666338, )

(666339, )

(666340, )

(666341, )

(668945, )

(668946, )

(668947, )

(669174, )

(669175, )

(669926, )

(669974, )

(669975, )

(669976, )

(674871, )

(674872, )

(674873, )

(674874, )

(674875, )

(675300, )

(675301, )

(675302, )

(683079, )

(683080, )

(683081, )

(684995, )

(684996, )

(684997, )

(684998, )

(691087, )

(691088, )

(691089, )

(696222, )

(698615, )

(702598, )

(702599, )

(702600, )

(702601, )

(700102, )

(700103, )

(7001


(166125, )

(166130, )

(168639, )

(168640, )

(168641, )

(168642, )

(168643, )

(168644, )

(168645, )

(169395, )

(169396, )

(169652, )

(169653, )

(171233, )

(171776, )

(171777, )

(171778, )

(171779, )

(172636, )

(172645, )

(180063, )

(180064, )

(180065, )

(180066, )

(180067, )

(180068, )

(180069, )

(193040, )

(193041, )

(193042, )

(193043, )

(193044, )

(193045, )

(200695, )

(200696, )

(200697, )

(202915, )

(202916, )

(202917, )

(202947, )

(202948, )

(202949, )

(208190, )

(208191, )

(208192, )

(208193, )

(208194, )

(210109, )

(210680, )

(210681, )

(216613, )

(216614, )

(216615, )

(216616, )

(216617, )

(216618, )

(216619, )

(223458, )

(223459, )

(223460, )

(223461, )

(223462, )

(229188, )

(229189, )

(229190, )

(229191, )

(229192, )

(231195, )

(231196, )

(231197, )

(231198, )

(280418, )

(280419, )

(280420, )

(280421, )

(280422, )

(280423, )

(280424, )

(280425, )

(286416, )

(286417, )

(286418, )

(286419, )

(28

(487483, )

(487484, )

(491286, )

(491287, )

(491288, )

(491289, )

(491290, )

(491291, )

(491292, )

(491293, )

(497892, )

(497893, )

(497894, )

(497895, )

(497896, )

(497897, )

(498973, )

(498974, )

(498975, )

(498976, )

(510871, )

(510872, )

(510873, )

(510874, )

(510875, )

(510876, )

(510877, )

(513948, )

(515230, )

(515231, )

(514372, )

(514373, )

(514374, )

(514375, )

(514376, )

(514377, )

(514378, )

(518121, )

(518122, )

(518123, )

(518124, )

(518125, )

(519010, )

(531894, )

(592880, )

(592881, )

(592882, )

(532195, )

(532196, )

(532197, )

(532198, )

(532199, )

(586411, )

(586414, )

(586415, )

(587727, )

(587728, )

(587729, )

(587730, )

(587731, )

(587732, )

(587733, )

(590619, )

(594681, )

(594682, )

(594683, )

(592911, )

(592912, )

(592913, )

(592914, )

(592915, )

(592916, )

(592917, )

(592918, )

(606677, )

(606678, )

(606679, )

(650465, )

(650466, )

(650467, )

(650468, )

(650469, )

(656922, )

(656

(807639, )

(807640, )

(807641, )

(807642, )

(807643, )

(808285, )

(808286, )

(813507, )

(813508, )

(813509, )

(813510, )

(813511, )

(814517, )

(814518, )

(814519, )

(814520, )

(814521, )

(814522, )

(814523, )

(814524, )

(814525, )

(818983, )

(819217, )

(819218, )

(830892, )

(830893, )

(830894, )

(830895, )

(830896, )

(830897, )

(830898, )

(830899, )

(830900, )

(830901, )

(833277, )

(833278, )

(834095, )

(834115, )

(834116, )

(834117, )

(834118, )

(843879, )

(843880, )

(843912, )

(870582, )

(870583, )

(870584, )

(870585, )

(870586, )

(870587, )

(870588, )

(870589, )

(870590, )

(870591, )

(871360, )

(871361, )

(871362, )

(871363, )

(871364, )

(871365, )

(871366, )

(878813, )

(878814, )

(878815, )

(878816, )

(878817, )

(880111, )

(882625, )

(882626, )

(882627, )

(882628, )

(882629, )

(888099, )

(888100, )

(888101, )

(888102, )

(888103, )

(891429, )

(891430, )

(891431, )

(891432, )

(891433, )

(903466, )

(903


(1252209, )

(1252210, )

(1252211, )

(1252212, )

(1252213, )

(1262427, )

(1262428, )

(1262429, )

(1262430, )

(1262431, )

(1262432, )

(1262433, )

(1262434, )

(1262435, )

(1262436, )

(1267673, )

(1267674, )

(1267675, )

(1267676, )

(1267677, )

(1267678, )

(1267679, )

(1267680, )

(1267681, )

(1267682, )

(1278386, )

(1278387, )

(1278388, )

(1278389, )

(1278390, )

(1278391, )

(1278392, )

(1278393, )

(1278394, )

(1278395, )

(1301052, )

(1301053, )

(1301054, )

(1301055, )

(1301056, )

(1301057, )

(1303038, )

(1303039, )

(1303040, )

(1303041, )

(1303042, )

(1303043, )

(1304565, )

(1304566, )

(1304567, )

(1304568, )

(1304569, )

(1304570, )

(1308576, )

(1308577, )

(1308578, )

(1308579, )

(1308580, )

(1308581, )

(1308990, )

(1308991, )

(1308992, )

(1308993, )

(1308994, )

(1308995, )

(1311898, )

(1311899, )

(1311900, )

(1311901, )

(1311902, )

(1311903, )

(1311904, )

(1311905, )

(1329540, )

(1329541, )

(1329542, )

(1333369, )


(1292410, )

(1215482, )

(1215483, )

(1291358, )

(1216110, )

(1216111, )

(1221970, )

(1216712, )

(1216713, )

(1217869, )

(1219207, )

(1219208, )

(1219209, )

(1219210, )

(1219211, )

(1221376, )

(1221389, )

(1221390, )

(1221391, )

(1221392, )

(1221393, )

(1223044, )

(1241778, )

(1241779, )

(1223312, )

(1283719, )

(1283720, )

(1223602, )

(1276301, )

(1276302, )

(1223798, )

(1271734, )

(1271735, )

(1271736, )

(1271737, )

(1224260, )

(1259646, )

(1259647, )

(1259648, )

(1259649, )

(1224476, )

(1229063, )

(1229064, )

(1229065, )

(1226260, )

(1226261, )

(1226262, )

(1226263, )

(1226264, )

(1240870, )

(1240871, )

(1240872, )

(1240873, )

(1242170, )

(1242171, )

(1242172, )

(1242173, )

(1245537, )

(1245538, )

(1245539, )

(1245557, )

(1247496, )

(1252969, )

(1252970, )

(1252971, )

(1252972, )

(1252973, )

(1247641, )

(1249283, )

(1249284, )

(1249285, )

(1268932, )

(1268933, )

(1268934, )

(1268935, )

(1268936, )

(1268937, )

(217792, )

(227568, )

(227569, )

(227570, )

(227571, )

(239870, )

(239871, )

(239872, )

(239873, )

(239874, )

(239875, )

(280481, )

(280482, )

(280483, )

(280484, )

(280485, )

(280486, )

(280487, )

(280919, )

(280920, )

(280921, )

(280922, )

(280923, )

(280924, )

(280925, )

(280926, )

(287373, )

(287374, )

(287375, )

(287376, )

(287377, )

(287378, )

(287379, )

(287380, )

(292591, )

(292592, )

(292593, )

(292594, )

(292595, )

(297333, )

(297334, )

(297335, )

(297336, )

(297339, )

(304038, )

(304082, )

(304083, )

(323588, )

(323589, )

(323590, )

(323591, )

(330365, )

(330366, )

(330367, )

(330368, )

(330369, )

(332539, )

(332540, )

(332541, )

(332542, )

(333401, )

(333402, )

(333403, )

(333404, )

(335695, )

(335696, )

(335697, )

(335698, )

(339601, )

(339602, )

(339603, )

(339604, )

(340054, )

(340055, )

(340056, )

(340057, )

(344736, )

(344737, )

(344738, )

(344739, )

(410717, )

(410718, )

(410719, )

(410

(965199, )

(965650, )

(965651, )

(965652, )

(965653, )

(965654, )

(965655, )

(965656, )

(965657, )

(965658, )

(965659, )

(1011030, )

(1011031, )

(1011032, )

(1011033, )

(1011034, )

(1011035, )

(1032123, )

(1032124, )

(1032125, )

(1032126, )

(1032127, )

(1032128, )

(1032129, )

(1032130, )

(1040198, )

(1040199, )

(1040200, )

(1040201, )

(1040202, )

(1042390, )

(1042391, )

(1042392, )

(1042393, )

(1042394, )

(1044424, )

(1044425, )

(1044426, )

(1044427, )

(1044428, )

(1048399, )

(1048400, )

(1048401, )

(1048792, )

(1048793, )

(1048794, )

(1055651, )

(1055652, )

(1090320, )

(1090321, )

(1090322, )

(1090323, )

(1090433, )

(1090434, )

(1090435, )

(1090436, )

(1100014, )

(1100015, )

(1100016, )

(1104384, )

(1104385, )

(1104386, )

(1104387, )

(1104388, )

(1104389, )

(1104390, )

(1104627, )

(1104628, )

(1104629, )

(1104630, )

(1104631, )

(1104632, )

(1104633, )

(1108010, )

(1108011, )

(1108012, )

(1108013, )

(1108014, 

In [16]:
with connect_to_DB() as conn:
    df = table_to_DataFrame(conn, "Departments", until=5000)

    print(df)

     DepartmentID                            Department Mnemonic  InputID  \
0               0                      (niet toegekend)     NONE        0   
1               1                        Japan en Korea    JAPKO        0   
2               2                          China- regio    CHINA        0   
3               3                Insulair Zuidoost-Azië    INSUL        0   
4               4  Zuid-Azië en vasteland Zuidoost-Azië    ZAZIE        0   
..            ...                                   ...      ...      ...   
385            63                   Museum Puri Lukisan      MPL        0   
386            96                        (not assigned)     NONE        0   
387            97                   Museum Puri Lukisan      MPL        0   
388            98                          Aztec-extern    AZTEC        0   
389            99                          Aztec-extern    AZTEC        0   

       LoginID             EnteredDate  NumRandomObjs  DefaultFormID  \
0  